In [ ]:
import pandas as pd
!pip install kaggle
!unzip jigsaw-toxic-comment-classification-challenge.zip
import os
import zipfile

unzip:  cannot find or open jigsaw-toxic-comment-classification-challenge.zip, jigsaw-toxic-comment-classification-challenge.zip.zip or jigsaw-toxic-comment-classification-challenge.zip.ZIP.


In [ ]:
import os
import zipfile
import pandas as pd
# Load the dataset
if not os.path.exists('train.csv'):
    if os.path.exists('train.csv.zip'):
        with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
            zip_ref.extractall()
        print("The file train.csv has been successfully extracted.")
    else:
        print("The file train.csv or train.csv.zip was not found.")
else:
    print("The file train.csv already exists in the current directory.")

The file train.csv has been successfully extracted.


In [ ]:
# # Information of the dataset
# data = pd.read_csv('train.csv')
# print("\nDataset information:")
# print(data.info())

In [ ]:
data = pd.read_csv('train.csv')
# Defining class columns
class_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Calculating the number of labels in each class
class_counts = data[class_columns].sum()

# Analyzing comments with no labels
no_labels_count = (data[class_columns].sum(axis=1) == 0).sum()

# Calculating percentage distribution
total_comments = len(data)
class_percentages = class_counts / total_comments * 100
no_labels_percentage = no_labels_count / total_comments * 100

# Formatted output
print("Distribution among classes and without labels:")
print(f"{'Class':<15}{'Count':<15}{'Percentage':<10}")
print("-" * 40)
for label, count in class_counts.items():
    percentage = class_percentages[label]
    print(f"{label:<15}{count:<15}{percentage:<10.2f}%")
print(f"{'Non-toxic':<15}{no_labels_count:<15}{no_labels_percentage:<10.2f}%")

Distribution among classes and without labels:
Class          Count          Percentage
----------------------------------------
toxic          15294          9.58      %
severe_toxic   1595           1.00      %
obscene        8449           5.29      %
threat         478            0.30      %
insult         7877           4.94      %
identity_hate  1405           0.88      %
Non-toxic      143346         89.83     %


In [ ]:

import re
from string import punctuation
from collections import Counter

In [ ]:
# Function to clean text for use with BERT
def clean_text_for_bert(text):
    # Remove newline characters
    text = re.sub(r'\n', ' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove time in the format HH:MM
    text = re.sub(r'\b\d{1,2}:\d{2}\b', '', text)
    # Remove dates.
    text = re.sub(r'\b\d{4}[-/\.]\d{2}[-/\.]\d{2}\b', '', text)
    text = re.sub(r'\b\d{2}[-/\.]\d{2}[-/\.]\d{4}\b', '', text)
    # Remove IP addresses
    text = re.sub(r'\b\d{1,3}(?:\.\d{1,3}){3}\b', '', text)
    return text

# Create a new column for cleaned data
data['cleaned_comment_text_for_bert'] = data['comment_text'].apply(clean_text_for_bert)
# Check the result
print("Original text:")
print(data['comment_text'].iloc[0])
print("\nCleaned text:")
print(data['cleaned_comment_text_for_bert'].iloc[0])

Original text:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Cleaned text:
Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.


In [ ]:
# Install the required libraries
!pip install transformers torch
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer


In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Define the class columns
class_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Add a column that indicates whether a comment is toxic (1 - toxic, 0 - non-toxic)
data['is_toxic'] = (data[class_columns].sum(axis=1) > 0).astype(int)

# Split features and labels
X = data['cleaned_comment_text_for_bert'].values.reshape(-1, 1)
y = data['is_toxic'].values

# Perform undersampling of non-toxic (0) to match the level of toxic (1)
rus = RandomUnderSampler(sampling_strategy=1.0, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

# Create a DataFrame with undersampled comments
balanced_data = pd.DataFrame(X_resampled, columns=['cleaned_comment_text_for_bert'])
balanced_data['is_toxic'] = y_resampled

# Restore the original labels for each comment
for col in class_columns:
    balanced_data[col] = 0
    balanced_data.loc[balanced_data['is_toxic'] == 1, col] = data.loc[data['is_toxic'] == 1, col].values

# Remove the temporary column
balanced_data.drop(columns=['is_toxic'], inplace=True)




/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [ ]:
# Checking class distribution in balanced_data
class_counts_balanced = balanced_data[class_columns].sum()
no_labels_count_balanced = (balanced_data[class_columns].sum(axis=1) == 0).sum()

# Calculating percentage distribution
total_comments_balanced = len(balanced_data)
class_percentages_balanced = class_counts_balanced / total_comments_balanced * 100
no_labels_percentage_balanced = no_labels_count_balanced / total_comments_balanced * 100

# Displaying results
print("Distribution among classes in balanced_data:")
print(f"{'Class':<15}{'Count':<15}{'Percentage':<10}")
print("-" * 40)
for label, count in class_counts_balanced.items():
    percentage = class_percentages_balanced[label]
    print(f"{label:<15}{count:<15}{percentage:<10.2f}%")
print(f"{'Non-toxic':<15}{no_labels_count_balanced:<15}{no_labels_percentage_balanced:<10.2f}%")



Distribution among classes in balanced_data:
Class          Count          Percentage
----------------------------------------
toxic          15294          47.13     %
severe_toxic   1595           4.92      %
obscene        8449           26.04     %
threat         478            1.47      %
insult         7877           24.27     %
identity_hate  1405           4.33      %
Non-toxic      16225          50.00     %


In [ ]:
# import numpy as np
# import pandas as pd
# # Визначення класів
# class_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# # 1. Визначаємо non-toxic коментарі (мітки == 0 по всіх класах)
# non_toxic_mask = (data[class_columns].sum(axis=1) == 0)  # non-toxic якщо сума == 0
# X_non_toxic = data.loc[non_toxic_mask, 'cleaned_comment_text_for_bert'].values
# y_non_toxic = data.loc[non_toxic_mask, class_columns].values

# # 2. Зменшуємо кількість non-toxic коментарів до 15600
# num_samples = 15600
# if len(X_non_toxic) > num_samples:
#     indices = np.random.choice(len(X_non_toxic), num_samples, replace=False)
#     X_non_toxic_resampled = X_non_toxic[indices]
#     y_non_toxic_resampled = y_non_toxic[indices]
# else:
#     X_non_toxic_resampled = X_non_toxic
#     y_non_toxic_resampled = y_non_toxic

# # 3. Створюємо DataFrame з undersampled non-toxic коментарями
# balanced_non_toxic = pd.DataFrame(X_non_toxic_resampled, columns=['cleaned_comment_text_for_bert'])
# balanced_non_toxic[class_columns] = y_non_toxic_resampled

# # 4. Об'єднуємо збалансовані non-toxic з усіма іншими коментарями (токсичними та non-toxic)
# balanced_data = pd.concat([
#     balanced_non_toxic,        # undersampled non-toxic
#     data[~non_toxic_mask]      # всі токсичні коментарі та non-toxic, які залишилися
# ])

# # 5. Скидання індексації
# balanced_data.reset_index(drop=True, inplace=True)

# print(f"Кількість коментарів після undersampling: {balanced_data.shape[0]}")




Кількість коментарів після undersampling: 31825


In [ ]:
!pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 20.5 MB/s eta 0:00:00


In [ ]:
!pip install nltk
import nlpaug.augmenter.word as naw
import nltk
# nltk.download('all')

# Download necessary NLTK packages
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

nltk.download('averaged_perceptron_tagger_eng')
# Create synonym augmenter
syn_aug = naw.SynonymAug(aug_src='wordnet')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [ ]:
# 1. Count the number of comments in each class
class_counts = balanced_data[class_columns].sum()

# 2. Analyze comments that belong to only one class
single_class_counts = {}
multi_label_counts = {}

for label in class_columns:
    # Select comments where only one class = 1, and others = 0
    single_class = balanced_data[(balanced_data[label] == 1) & (balanced_data[class_columns].sum(axis=1) == 1)]
    single_class_counts[label] = len(single_class)

    # Select comments where the label is part of a multilabel
    multi_class = balanced_data[(balanced_data[label] == 1) & (balanced_data[class_columns].sum(axis=1) > 1)]
    multi_label_counts[label] = len(multi_class)

    # Analyze other labels for multilabel
    print(f"\n Analysis for class '{label}':")
    print(f"Only {label}: {len(single_class)}")
    print(f"{label} + other labels: {len(multi_class)}")
    if len(multi_class) > 0:
        print("Label combinations:")
        print(multi_class[class_columns].sum().sort_values(ascending=False))

# 3. Output the summary table
analysis_df = pd.DataFrame({
    'Total': class_counts,
    'Single Class': pd.Series(single_class_counts),
    'Multi-label': pd.Series(multi_label_counts)
})

# 4. Display results
print("\n Class distribution analysis:")
print(analysis_df)

# Percentage of multi-label comments for each class
analysis_df['Multi-label %'] = (analysis_df['Multi-label'] / analysis_df['Total']) * 100

print("\n Percentage of multi-label comments for each class:")
print(analysis_df[['Multi-label %']])



 Analysis for class 'toxic':
Only toxic: 5666
toxic + other labels: 9628
Label combinations:
toxic            9628
obscene          7926
insult           7344
severe_toxic     1595
identity_hate    1302
threat            449
dtype: int64

 Analysis for class 'severe_toxic':
Only severe_toxic: 0
severe_toxic + other labels: 1595
Label combinations:
toxic            1595
severe_toxic     1595
obscene          1517
insult           1371
identity_hate     313
threat            112
dtype: int64

 Analysis for class 'obscene':
Only obscene: 317
obscene + other labels: 8132
Label combinations:
obscene          8132
toxic            7926
insult           6155
severe_toxic     1517
identity_hate    1032
threat            301
dtype: int64

 Analysis for class 'threat':
Only threat: 22
threat + other labels: 456
Label combinations:
threat           456
toxic            449
insult           307
obscene          301
severe_toxic     112
identity_hate     98
dtype: int64

 Analysis for class 'insul

In [ ]:
# 1. Count the total number of comments in each class
total_class_counts = balanced_data[class_columns].sum()

# 2. Initialize a DataFrame to store the relationships between classes
relation_matrix = pd.DataFrame(0, index=class_columns, columns=class_columns)

# 3. Analyze relationships between classes
for label in class_columns:
    # Select all comments that belong to the class label
    class_data = balanced_data[balanced_data[label] == 1]

    # Count the number of comments that also belong to other classes
    relation_counts = class_data[class_columns].sum()

    # Fill in the relationship matrix (excluding itself)
    relation_matrix.loc[label] = relation_counts

# 4. Normalize for relationships (in percentage)
relation_percentage = (relation_matrix.div(total_class_counts, axis=1) * 100).round(2)

# 5. Display results
print(" Relationship matrix between classes (number of comments):")
print(relation_matrix)

print("\n Relationship matrix between classes (percentages):")
print(relation_percentage)


 Relationship matrix between classes (number of comments):
               toxic  severe_toxic  obscene  threat  insult  identity_hate
toxic          15294          1595     7926     449    7344           1302
severe_toxic    1595          1595     1517     112    1371            313
obscene         7926          1517     8449     301    6155           1032
threat           449           112      301     478     307             98
insult          7344          1371     6155     307    7877           1160
identity_hate   1302           313     1032      98    1160           1405

 Relationship matrix between classes (percentages):
                toxic  severe_toxic  obscene  threat  insult  identity_hate
toxic          100.00        100.00    93.81   93.93   93.23          92.67
severe_toxic    10.43        100.00    17.95   23.43   17.41          22.28
obscene         51.82         95.11   100.00   62.97   78.14          73.45
threat           2.94          7.02     3.56  100.00    3.9

In [ ]:
# from sklearn.utils import resample
# import nlpaug.augmenter.word as naw
# import pandas as pd

# Initialize the synonym augmenter
syn_aug = naw.SynonymAug(aug_src='wordnet')

# Text augmentation function
def augment_text(text, n=1):
    augmented_texts = [text]
    for _ in range(n):
        augmented_texts.append(syn_aug.augment(text))
    return augmented_texts

# 1. First augmentation for severe_toxic, threat, identity_hate

augmented_rows = []

for label in ['severe_toxic', 'threat', 'identity_hate']:
    class_data = balanced_data[balanced_data[label] == 1]
    print(f"\nNumber of {label} before augmentation: {len(class_data)}")

    n = 6 if label == 'threat' else 1

    for _, row in class_data.iterrows():
        augmented_comments = augment_text(row['cleaned_comment_text_for_bert'], n=n)

        for comment in augmented_comments:
            augmented_rows.append({
                'cleaned_comment_text_for_bert': comment,
                'toxic': 0,
                'severe_toxic': row['severe_toxic'],
                'obscene': row['obscene'],
                'threat': row['threat'],
                'insult': row['insult'],
                'identity_hate': row['identity_hate']
            })

# 2. Create DataFrame and merge

if augmented_rows:
    augmented_df = pd.DataFrame(augmented_rows)
    balanced_data = pd.concat([balanced_data, augmented_df], ignore_index=True)
    print(f"\nAdded {len(augmented_df)} new comments.")
else:
    print("Augmentation did not create new texts.")


Кількість severe_toxic до аугментації: 1595

Кількість threat до аугментації: 478

Кількість identity_hate до аугментації: 1405

Added 9346 new comments.


In [ ]:
# print("\nРозподіл класів після аугментації:")
# print(balanced_data[class_columns].sum())
# print("\nShape of balanced_data after augmentation:", balanced_data.shape)


Розподіл класів після аугментації:
toxic            15294
severe_toxic      6195
obscene          15654
threat            4244
insult           15088
identity_hate     5527
dtype: int64

Shape of balanced_data after augmentation: (41796, 7)


In [ ]:
# Checking class distribution in balanced_data
class_counts_balanced = balanced_data[class_columns].sum()
no_labels_count_balanced = (balanced_data[class_columns].sum(axis=1) == 0).sum()

# Calculating percentage distribution
total_comments_balanced = len(balanced_data)
class_percentages_balanced = class_counts_balanced / total_comments_balanced * 100
no_labels_percentage_balanced = no_labels_count_balanced / total_comments_balanced * 100

# Displaying results
print("Distribution among classes in balanced_data:")
print(f"{'Class':<15}{'Count':<15}{'Percentage':<10}")
print("-" * 40)
for label, count in class_counts_balanced.items():
    percentage = class_percentages_balanced[label]
    print(f"{label:<15}{count:<15}{percentage:<10.2f}%")
print(f"{'Non-toxic':<15}{no_labels_count_balanced:<15}{no_labels_percentage_balanced:<10.2f}%")



Distribution among classes in balanced_data:
Class          Count          Percentage
----------------------------------------
toxic          15294          36.59     %
severe_toxic   6195           14.82     %
obscene        15654          37.45     %
threat         4244           10.15     %
insult         15088          36.10     %
identity_hate  5527           13.22     %
Non-toxic      16225          38.82     %


In [ ]:
import numpy as np

augmented_rows_second = []

for label in ['severe_toxic', 'threat', 'identity_hate']:
    class_data = balanced_data[balanced_data[label] == 1]

    for _, row in class_data.iterrows():
        # Set augmentation probabilities for each class
        if label in ['severe_toxic', 'identity_hate']:
            n = np.random.choice([0, 1], p=[0.5, 0.5])
        else:
            n = 1

        # Perform text augmentation
        augmented_comments = augment_text(row['cleaned_comment_text_for_bert'], n=n)

        for comment in augmented_comments:
            augmented_rows_second.append({
                'cleaned_comment_text_for_bert': comment,
                'toxic': 0,
                'severe_toxic': 1 if label == 'severe_toxic' else 0,
                'obscene': 0,
                'threat': 1 if label == 'threat' else 0,
                'insult': 0,
                'identity_hate': 1 if label == 'identity_hate' else 0
            })

# Create DataFrame from augmented texts
if augmented_rows_second:
    augmented_df_second = pd.DataFrame(augmented_rows_second)
    balanced_data = pd.concat([balanced_data, augmented_df_second], ignore_index=True)
    print(f"\nAdded {len(augmented_df_second)} new comments (with zeroing of other classes).")
else:
    print("Augmentation did not create new texts.")


# Count after the second augmentation

print("\nClass distribution after second augmentation:")
print(balanced_data[class_columns].sum())
print("\nShape of balanced_data after second augmentation:", balanced_data.shape)

# Output the number of new comments for each class
aug_count = augmented_df_second[['severe_toxic', 'threat', 'identity_hate']].sum()
print("\nNumber of new augmented comments by class:")
print(aug_count)

# # Check examples of new comments
# print("\nSample of augmented comments (second run):")
# print(augmented_df_second[['cleaned_comment_text_for_bert', 'toxic', 'severe_toxic', 'threat', 'obscene', 'insult', 'identity_hate']].head(10))



Added 26082 new comments (with zeroing of other classes).

Class distribution after second augmentation:
toxic            15294
severe_toxic     15500
obscene          15654
threat           12732
insult           15088
identity_hate    13816
dtype: int64

Shape of balanced_data after second augmentation: (67878, 7)

Number of new augmented comments by class:
severe_toxic     9305
threat           8488
identity_hate    8289
dtype: int64


In [ ]:
# Checking class distribution in balanced_data
class_counts_balanced = balanced_data[class_columns].sum()
no_labels_count_balanced = (balanced_data[class_columns].sum(axis=1) == 0).sum()

# Calculating percentage distribution
total_comments_balanced = len(balanced_data)
class_percentages_balanced = class_counts_balanced / total_comments_balanced * 100
no_labels_percentage_balanced = no_labels_count_balanced / total_comments_balanced * 100

# Displaying results
print("Distribution among classes in balanced_data:")
print(f"{'Class':<15}{'Count':<15}{'Percentage':<10}")
print("-" * 40)
for label, count in class_counts_balanced.items():
    percentage = class_percentages_balanced[label]
    print(f"{label:<15}{count:<15}{percentage:<10.2f}%")
print(f"{'Non-toxic':<15}{no_labels_count_balanced:<15}{no_labels_percentage_balanced:<10.2f}%")



Distribution among classes in balanced_data:
Class          Count          Percentage
----------------------------------------
toxic          15294          22.53     %
severe_toxic   15500          22.84     %
obscene        15654          23.06     %
threat         12732          18.76     %
insult         15088          22.23     %
identity_hate  13816          20.35     %
Non-toxic      16225          23.90     %


In [ ]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Function for tokenizing text using the BERT tokenizer
def tokenize_text(text, max_len=128):
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)


In [ ]:
# Converts label values into a PyTorch tensor
def encode_labels(row):
    return torch.tensor([row['toxic'], row['severe_toxic'], row['obscene'], row['threat'], row['insult'], row['identity_hate']])


In [ ]:
# Function to prepare data for the model
def prepare_data(data, max_len=128):
    input_ids_list = []
    attention_masks_list = []
    labels_list = []

    for _, row in data.iterrows():
        # Tokenizing the text
        input_ids, attention_mask = tokenize_text(row['cleaned_comment_text_for_bert'], max_len)
        input_ids_list.append(input_ids)
        attention_masks_list.append(attention_mask)

        # Converting labels
        labels = torch.tensor(row[class_columns].astype(int).values)
        labels_list.append(labels)


     # Creating the final DataFrame
    return pd.DataFrame({
        "input_ids": input_ids_list,
        "attention_mask": attention_masks_list,
        "labels": labels_list
    })

In [ ]:
prepared_data = prepare_data(balanced_data, max_len=128)
print(prepared_data.head())

                                           input_ids  \
0  [tensor(101), tensor(1000), tensor(2821), tens...   
1  [tensor(101), tensor(2024), tensor(2017), tens...   
2  [tensor(101), tensor(25430), tensor(4140), ten...   
3  [tensor(101), tensor(3104), tensor(2061), tens...   
4  [tensor(101), tensor(1052), tensor(1012), tens...   

                                      attention_mask  \
0  [tensor(1), tensor(1), tensor(1), tensor(1), t...   
1  [tensor(1), tensor(1), tensor(1), tensor(1), t...   
2  [tensor(1), tensor(1), tensor(1), tensor(1), t...   
3  [tensor(1), tensor(1), tensor(1), tensor(1), t...   
4  [tensor(1), tensor(1), tensor(1), tensor(1), t...   

                                              labels  
0  [tensor(0), tensor(0), tensor(0), tensor(0), t...  
1  [tensor(0), tensor(0), tensor(0), tensor(0), t...  
2  [tensor(0), tensor(0), tensor(0), tensor(0), t...  
3  [tensor(0), tensor(0), tensor(0), tensor(0), t...  
4  [tensor(0), tensor(0), tensor(0), tensor(0), t..

In [ ]:
# Convert columns to tensors
input_ids_tensor = torch.stack(prepared_data['input_ids'].tolist())
attention_mask_tensor = torch.stack(prepared_data['attention_mask'].tolist())
labels_tensor = torch.stack(prepared_data['labels'].tolist())

# Check shapes
print(f"Shape of input_ids: {input_ids_tensor.shape}")
print(f"Shape of attention_mask: {attention_mask_tensor.shape}")
print(f"Shape of labels: {labels_tensor.shape}")

Shape of input_ids: torch.Size([67878, 128])
Shape of attention_mask: torch.Size([67878, 128])
Shape of labels: torch.Size([67878, 6])


In [ ]:
# Create a DataFrame for BERT processing
bert_data = pd.DataFrame({
    'input_ids': input_ids_tensor.tolist(),
    'attention_mask': attention_mask_tensor.tolist(),
    'labels': labels_tensor.tolist()
})

# Save as CSV format
bert_data.to_csv('bert_ready_data2.csv', index=False)
print("DataFrame saved as 'bert_ready_data.csv'.")

DataFrame saved as 'bert_ready_data.csv'.


In [ ]:
# loaded_data = pd.read_csv('bert_ready_data2.csv')
# print(loaded_data.head())